In [50]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [42]:
class DecisionTree:
    def __init__(self, max_depth=10, min_split=2):
        self.max_depth = max_depth
        self.min_split = min_split

    def fit(self, X, y):
        self.labels = np.unique(y)
        self.root = self.constructTree(X, y, 0)

    def predict(self, X):
        ypred = []
        for i in range(len(X)):
            p = self.predict1(X[i], self.root)
            lb = p  # Now correctly returns class label
            ypred.append(lb)
        return np.array(ypred)

    def predict1(self, t, node):
        if node['isLeaf']:
            return node['prediction']  # Now returns a class label, not probabilities
        elif t[node['Feature']] < node['Threshold']:
            return self.predict1(t, node['Left'])
        else:
            return self.predict1(t, node['Right'])
        
    def constructTree(self, X, y, curDepth=0):
        if len(X) == 0 or curDepth >= self.max_depth:
            return {"isLeaf": True, "prediction": self.most_common_class(y)}

        if len(y) <= self.min_split:
            return {"isLeaf": True, "prediction": self.most_common_class(y)}

        nf = X.shape[1]  # Number of features
        parentEntropy = self.entropy(y)
        weightedChildEntropy = []

        for feat in range(nf):
            th = X[:, feat].mean()
            left_mask = X[:, feat] < th
            right_mask = ~left_mask
            wec = (
                np.mean(left_mask) * self.entropy(y[left_mask])
                + np.mean(right_mask) * self.entropy(y[right_mask])
            )
            weightedChildEntropy.append(wec)

        InformationGain = parentEntropy - np.array(weightedChildEntropy)
        feat = np.argmax(InformationGain)
        th = X[:, feat].mean()

        node = {
            "Feature": feat,
            "Threshold": th,
            "isLeaf": False,
            "Left": None,
            "Right": None
        }

        left_mask = X[:, feat] < th
        right_mask = ~left_mask

        if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
            return {"isLeaf": True, "prediction": self.most_common_class(y)}

        node["Left"] = self.constructTree(X[left_mask], y[left_mask], curDepth + 1)
        node["Right"] = self.constructTree(X[right_mask], y[right_mask], curDepth + 1)

        return node

    def most_common_class(self, y):
        if len(y) == 0:
            return 0  # Default to class 0 if no data is present
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]
        
    def entropy(self, y):
        if len(y) == 0:
            return 0
        values, counts = np.unique(y, return_counts=True)
        prob = counts / len(y)
        return -np.sum(np.where(prob == 0, 0, prob * np.log2(prob)))  # Avoids log(0)

    def accuracy(self, y_true, y_pred):
        return np.mean(y_true == y_pred) * 100



In [43]:
data = pd.read_csv('Titanic-Dataset.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:
cols_drop = ["PassengerId", "Name", "Ticket", "Cabin"]
data = data.drop(cols_drop, axis=1)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [45]:
data["Embarked"] = data["Embarked"].fillna(data["Embarked"].mode()[0])
data["Age"] = data["Age"].fillna(data["Age"].mean())

In [46]:
lab_enc = LabelEncoder()
data["Sex"] = lab_enc.fit_transform(data["Sex"])
data["Embarked"] = lab_enc.fit_transform(data["Embarked"])
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [52]:
input_cols = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
output_col = "Survived"

X_data = np.array(data[input_cols])
Y_data = np.array(data[output_col])
scaler = StandardScaler()
X_data[:, [2, 5]] = scaler.fit_transform(X_data[:, [2, 5]])
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [84]:
x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.8, random_state=42)

In [85]:
dt = DecisionTree(max_depth=10, min_split=2)
dt.fit(x_train, y_train)
pred = dt.predict(x_test)
acc = dt.accuracy(y_test, pred)
print("Accuracy:", round(acc, 3))

Accuracy: 83.799


Descision Tree using sklearn

In [86]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [87]:
dt = DecisionTreeClassifier(max_depth=7)
dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=7)

In [88]:
accuracy_score(y_train,dt.predict(x_train))

0.8820224719101124

In [89]:
accuracy_score(y_test,dt.predict(x_test))

0.8044692737430168